# 1. Limpieza y normalizacion y exploracion inicial

## Objetivo
Familiarizarse con los datos de transacciones y entender su estructura:
- Shape, tipos de datos, valores nulos, valores atípicos
- Distribuciones de montos y tipos de transacciones
- Identificación de patrones iniciales
- Limpiar las columnas y organziar el df

## Dataset
- **Archivos**: `sample_data_0006_part_00.parquet` y `sample_data_0007_part_00.parquet`
- **Ubicación**: `../data/`

In [4]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path

# Configuración para visualizaciones
plt.style.use('default')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Configuración de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("✅ Librerías importadas correctamente")

✅ Librerías importadas correctamente


## 1. Carga de Datos

Vamos a cargar los dos archivos parquet y verificar si tienen la misma estructura para poder unirlos.

In [5]:
# Definir rutas de los archivos
data_path = Path("../data/")
file1 = data_path / "sample_data_0006_part_00.parquet"
file2 = data_path / "sample_data_0007_part_00.parquet"

# Cargar cada archivo por separado para explorar
print("📁 Cargando archivos...")
df1 = pd.read_parquet(file1)
df2 = pd.read_parquet(file2)

print(f"✅ Archivo 1 cargado: {file1.name}")
print(f"   Shape: {df1.shape}")
print(f"✅ Archivo 2 cargado: {file2.name}")
print(f"   Shape: {df2.shape}")

📁 Cargando archivos...
✅ Archivo 1 cargado: sample_data_0006_part_00.parquet
   Shape: (10758418, 8)
✅ Archivo 2 cargado: sample_data_0007_part_00.parquet
   Shape: (10758500, 8)


In [6]:
# Verificar estructura de ambos archivos
print("🔍 Verificando estructura de los archivos...\n")

print("📋 COLUMNAS ARCHIVO 1:")
print(f"Columnas: {list(df1.columns)}\n")

print("📋 COLUMNAS ARCHIVO 2:")
print(f"Columnas: {list(df2.columns)}\n")

# Verificar si las columnas son iguales
columnas_iguales = list(df1.columns) == list(df2.columns)
print(f"¿Las columnas son iguales? {columnas_iguales}")

if columnas_iguales:
    print("✅ Los archivos tienen la misma estructura, se pueden unir")
else:
    print("⚠️ Los archivos tienen diferente estructura")

🔍 Verificando estructura de los archivos...

📋 COLUMNAS ARCHIVO 1:
Columnas: ['merchant_id', '_id', 'subsidiary', 'transaction_date', 'account_number', 'user_id', 'transaction_amount', 'transaction_type']

📋 COLUMNAS ARCHIVO 2:
Columnas: ['merchant_id', '_id', 'subsidiary', 'transaction_date', 'account_number', 'user_id', 'transaction_amount', 'transaction_type']

¿Las columnas son iguales? True
✅ Los archivos tienen la misma estructura, se pueden unir


In [7]:
# Unir los datasets
if columnas_iguales:
    print("🔗 Uniendo los datasets...")
    df = pd.concat([df1, df2], ignore_index=True)
    print(f"✅ Dataset unificado creado")
    print(f"   Shape final: {df.shape}")
    print(f"   Registros archivo 1: {len(df1)}")
    print(f"   Registros archivo 2: {len(df2)}")
    print(f"   Total registros: {len(df)}")
    
    # Liberar memoria de los dataframes individuales
    del df1, df2
else:
    print("❌ No se pueden unir los archivos debido a diferencias en estructura")

🔗 Uniendo los datasets...
✅ Dataset unificado creado
   Shape final: (21516918, 8)
   Registros archivo 1: 10758418
   Registros archivo 2: 10758500
   Total registros: 21516918


## 2. Analisis y preprocesamiento del dataset

In [8]:
# Vista rápida e información general del dataset
print("📊 INFORMACIÓN GENERAL DEL DATASET")
print("=" * 60)
print(f"💾 Memoria utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print("\n📋 TIPOS DE DATOS:")
print("-" * 40)
tipo_counts = df.dtypes.value_counts()
for tipo, count in tipo_counts.items():
    print(f"  {tipo}: {count} columnas")

print(f"\n📝 NOMBRES DE LAS COLUMNAS:")
print("-" * 40)
for i, col in enumerate(df.columns, 1):
    print(f"  {i:2d}. {col} ({df[col].dtype})")

print(f"\nPRIMERAS 3 FILAS:")
print("-" * 40)
display(df.head(3))

print(f"\nÚLTIMAS 3 FILAS:")
print("-" * 40)
display(df.tail(3))

📊 INFORMACIÓN GENERAL DEL DATASET
💾 Memoria utilizada: 12234.13 MB

📋 TIPOS DE DATOS:
----------------------------------------
  object: 7 columnas
  datetime64[ns]: 1 columnas

📝 NOMBRES DE LAS COLUMNAS:
----------------------------------------
   1. merchant_id (object)
   2. _id (object)
   3. subsidiary (object)
   4. transaction_date (datetime64[ns])
   5. account_number (object)
   6. user_id (object)
   7. transaction_amount (object)
   8. transaction_type (object)

PRIMERAS 3 FILAS:
----------------------------------------


,merchant_id,_id,subsidiary,transaction_date,account_number,user_id,transaction_amount,transaction_type
0,075d178871d8d48502bf1f54887e52fe,aa8dacff663072244d0a8ab6bbe36b93,824b2af470cbe6a65b15650e03b740fc,2021-09-12 18:32:03,648e257c9d74909a1f61c54b93a9e1b3,ba42d192a145583ba8e7bf04875f837f,178.33365037,CREDITO
1,075d178871d8d48502bf1f54887e52fe,a53bb81bd0bba2ae2535bda7ea5a550c,2d8d34be7509a6b1262336d036fdb324,2021-09-12 18:31:58,c0b62f9046c83ea5543ea46a497a4d6e,5cfff960ea6d732c1ba3e63d24f3be52,35.66673007,CREDITO
2,075d178871d8d48502bf1f54887e52fe,79f893ea65c06fe2933f3847c88c272f,5eeb18254850b21af0a6bb2697913cd3,2021-09-12 18:31:56,872d10143fc0ac7d5de467806f6bef81,c97e63a92c82c7217b333635d75928ed,142.66692029,CREDITO



ÚLTIMAS 3 FILAS:
----------------------------------------


,merchant_id,_id,subsidiary,transaction_date,account_number,user_id,transaction_amount,transaction_type
21516915,838a8fa992a4aa2fb5a0cf8b15b63755,f19c70c14bae6f6c8d53c2a85f5f59f6,55fb0b01915693a7d5de686f9f4b1cb6,2021-11-30 11:01:07,90865ae9ab4d82a1feec13eee9a4a303,356058d6c9548879be6ef65a9733ea44,594.44550123,CREDITO
21516916,838a8fa992a4aa2fb5a0cf8b15b63755,8d43b7b55023ccae39735707a811f38f,55fb0b01915693a7d5de686f9f4b1cb6,2021-11-30 11:02:05,90865ae9ab4d82a1feec13eee9a4a303,356058d6c9548879be6ef65a9733ea44,172.38919535,CREDITO
21516917,838a8fa992a4aa2fb5a0cf8b15b63755,b15c20c071ab0df8fa15c71e4dd35e7d,189977b64fded8e9a37df9fc749fad5c,2021-11-29 14:07:30,528173dc0da13e78938c9e58c5cb3673,9b4a7ac86ed81ad330366e766c739808,47.55564009,CREDITO


In [9]:
# Estadísticas descriptivas básicas
print("📈 ESTADÍSTICAS DESCRIPTIVAS BÁSICAS")
print("=" * 60)

# Columnas categóricas/texto
text_cols = df.select_dtypes(include=['object', 'string']).columns
print(f"\n📝 Columnas de texto/categóricas encontradas: {len(text_cols)}")
if len(text_cols) > 0:
    print(f"Columnas: {list(text_cols)}")
    for col in text_cols:
        unique_count = df[col].nunique()
        print(f"\n  📊 {col}:")
        print(f"    Valores únicos: {unique_count:,}")
        print(f"    Tipo: {df[col].dtype}")
        if unique_count <= 20:
            print(f"    Valores: {sorted(df[col].unique())}")
        else:
            print(f"    Primeros 5 valores: {sorted(df[col].unique())[:5]}")
else:
    print("⚠️ No se encontraron columnas de texto")

📈 ESTADÍSTICAS DESCRIPTIVAS BÁSICAS

📝 Columnas de texto/categóricas encontradas: 7
Columnas: ['merchant_id', '_id', 'subsidiary', 'account_number', 'user_id', 'transaction_amount', 'transaction_type']

  📊 merchant_id:
    Valores únicos: 3
    Tipo: object
    Valores: ['075d178871d8d48502bf1f54887e52fe', '817d18cd3c31e40e9bff0566baae7758', '838a8fa992a4aa2fb5a0cf8b15b63755']

  📊 _id:
    Valores únicos: 21,516,901
    Tipo: object
    Primeros 5 valores: ['000000c00a3d728b6b5c1580b4816959', '000002ea806e97f97a6a3d5acd2855b2', '000003f2f33e6a45ce7483113c709aa1', '000004b9c3cb1548d4f9771458dc5582', '0000059697b8090d5079c6f3e475f68b']

  📊 subsidiary:
    Valores únicos: 16,833
    Tipo: object
    Primeros 5 valores: ['00015fd77a0f4d869bea31bb7244e375', '000b48cd239b8411eec69c69963cb5dd', '000ed83916ade2efe9c7ed9c81f6daa5', '0012d2042453597ebcf06140b0e4ec3f', '00130d1dfa59170a208840ea4004f423']

  📊 account_number:
    Valores únicos: 3,099,711
    Tipo: object
    Primeros 5 valores

## Revision y transformacion de los tipos de datos de cada columna

In [10]:
#Transformar columna transaction_date a tipo datetime
df['transaction_date'] = pd.to_datetime(df['transaction_date'], errors='coerce')

# Transformar columna transaction_amount a tipo numérico
df['transaction_amount'] = pd.to_numeric(df['transaction_amount'], errors='coerce')

#Transformar transaction_type a tipo categórico
df['transaction_type'] = df['transaction_type'].astype('category')

#Verificar nuevamente los tipos de datos
print("\n📋 TIPOS DE DATOS DESPUÉS DE TRANSFORMACIONES:")
print("-" * 40)
print(df.dtypes)


📋 TIPOS DE DATOS DESPUÉS DE TRANSFORMACIONES:
----------------------------------------
merchant_id                   object
_id                           object
subsidiary                    object
transaction_date      datetime64[ns]
account_number                object
user_id                       object
transaction_amount           float64
transaction_type            category
dtype: object


## Verificar valores nulos

In [11]:
print("\n🔍 VERIFICANDO VALORES NULOS")

nulos = df.isnull().sum()
print(f"Total de columnas con valores nulos: {nulos[nulos > 0].shape[0]}")


🔍 VERIFICANDO VALORES NULOS
Total de columnas con valores nulos: 0


## Verificar valores duplicados en IDs que deberían ser únicos (_id)

In [12]:
print("\n🔍 VERIFICANDO VALORES DUPLICADOS EN IDs")

duplicados = df['_id'].duplicated().sum()
if duplicados > 0:
    print(f"⚠️ Se encontraron {duplicados} registros duplicados en la columna '_id'")
else:
    print("✅ No se encontraron registros duplicados en la columna '_id'")
        


🔍 VERIFICANDO VALORES DUPLICADOS EN IDs
⚠️ Se encontraron 17 registros duplicados en la columna '_id'


In [13]:
# Revisar los registros duplicados en '_id' manualmente

# 0. Numero de registros totales en la columna '_id'
total_ids = df['_id']
print(f"\nTotal de registros en la columna '_id': {len(total_ids)}")

# 1. Obtener la lista de IDs que aparecen más de una vez
dup_ids = df.loc[df['_id'].duplicated(), '_id'].unique()
print(f"IDs duplicados encontrados ({len(dup_ids)}):\n", dup_ids)

# 2. Filtrar todas las filas con esos IDs para inspección
df_dups = df[df['_id'].isin(dup_ids)].sort_values('_id')
display(df_dups)


Total de registros en la columna '_id': 21516918
IDs duplicados encontrados (17):
 ['53f3b7bf5cf7c5509279f385eedca459' '01dd3dc328d6c7e40453c509311f3b74'
 'c52a54da9927f445e5f621d6dac29bf4' '52227fbe95311404092cbc8fbf2d320b'
 'b93a2bb461682a71f593049250e68a4e' '8f5418532506951085466a90e0359377'
 'd737f8d70694471f3061002db7fb2ac1' '243091645de25b8bb86d21d136a0269d'
 'b479a0b93f874544e2003316d0ead444' '7db6492c97f5ca0d9864ec32bda0183c'
 'bd471dba050aed4a5ee64b4da9c50663' 'bcfa23b49bda298316b3738878a7b769'
 'b448e42e5b5e884fb1ae8285c16b9470' 'e81671a3fdbeedbe7e3352fe175b4a6e'
 '0d0791af36d96b466d65636e90e53427' '25288e8d44ecd67907a8f2e3c8343ad6'
 'bdc2f3e73aa4e46f96cc0ed64b0907ed']


,merchant_id,_id,subsidiary,transaction_date,account_number,user_id,transaction_amount,transaction_type
709845,075d178871d8d48502bf1f54887e52fe,01dd3dc328d6c7e40453c509311f3b74,62c21974aa375cd2689a8e883520c9da,2021-09-17 16:19:28,1fdcb18a9c87137a828cf01e5505e165,ad5fade2b0f886d802c910ac58f69835,118.889100,CREDITO
709846,075d178871d8d48502bf1f54887e52fe,01dd3dc328d6c7e40453c509311f3b74,62c21974aa375cd2689a8e883520c9da,2021-09-17 16:19:27,1fdcb18a9c87137a828cf01e5505e165,ad5fade2b0f886d802c910ac58f69835,118.889100,CREDITO
21378370,838a8fa992a4aa2fb5a0cf8b15b63755,0d0791af36d96b466d65636e90e53427,e6a32882c99aab1d92ae297da67402ed,2021-08-24 14:58:49,8aff7eb98beb9a548f77084c4c918e6d,a92f26f05f04a760b44fbc3bad3c63c9,130.778010,CREDITO
21378369,838a8fa992a4aa2fb5a0cf8b15b63755,0d0791af36d96b466d65636e90e53427,e6a32882c99aab1d92ae297da67402ed,2021-08-24 14:58:49,8aff7eb98beb9a548f77084c4c918e6d,a92f26f05f04a760b44fbc3bad3c63c9,130.778010,CREDITO
11251792,075d178871d8d48502bf1f54887e52fe,243091645de25b8bb86d21d136a0269d,ec74b26befb0decad98a1679527c8740,2021-06-03 13:04:05,966fc01b4652738ba1d299277357939b,6dd128b5ccd1b920f6c7a80525f99165,71.333460,CREDITO
11164594,075d178871d8d48502bf1f54887e52fe,243091645de25b8bb86d21d136a0269d,ec74b26befb0decad98a1679527c8740,2021-06-03 13:04:05,966fc01b4652738ba1d299277357939b,6dd128b5ccd1b920f6c7a80525f99165,71.333460,CREDITO
21394428,838a8fa992a4aa2fb5a0cf8b15b63755,25288e8d44ecd67907a8f2e3c8343ad6,5b526b634874a82bba8a08fe47087da3,2021-08-04 16:55:02,79307eeb61bde70c1fc6b601d5ea99d9,51641aec649e32ea4d224a49c2c1055e,154.555830,CREDITO
21394427,838a8fa992a4aa2fb5a0cf8b15b63755,25288e8d44ecd67907a8f2e3c8343ad6,5b526b634874a82bba8a08fe47087da3,2021-08-04 16:55:03,79307eeb61bde70c1fc6b601d5ea99d9,51641aec649e32ea4d224a49c2c1055e,154.555830,CREDITO
1156450,075d178871d8d48502bf1f54887e52fe,52227fbe95311404092cbc8fbf2d320b,024b459a0851f51d731099ac10d1b30a,2021-08-26 14:58:58,8495c31cd0ba9c1c6ccf493cdf0e3558,6405b4845df13af0ca2f03714ff486c2,475.556401,CREDITO
1156451,075d178871d8d48502bf1f54887e52fe,52227fbe95311404092cbc8fbf2d320b,024b459a0851f51d731099ac10d1b30a,2021-08-26 14:58:58,8495c31cd0ba9c1c6ccf493cdf0e3558,6405b4845df13af0ca2f03714ff486c2,475.556401,CREDITO


Despues de revisara a detalle NO es un “doble pago” real, sino un retry o duplicación en la capa de ingestión del banco.
Todas tienen la misma fecha y hora a excepción de una que tiene un segundo de diferencia.
Por lo tanto, podemos eliminarlas sin problema el primer registro de cada ID duplicado.

In [14]:
print("\n🔄 ELIMINANDO REGISTROS DUPLICADOS EN '_id'")

df = df.drop_duplicates(subset=['_id'], keep='first').reset_index(drop=True)
print(f"✅ Duplicados de _id eliminados. Nuevo shape: {df.shape}")

# Verificar nuevamente los valores duplicados
duplicados = df['_id'].duplicated().sum()
print(f"Total de registros duplicados en '_id' después de limpieza: {duplicados}")


🔄 ELIMINANDO REGISTROS DUPLICADOS EN '_id'
✅ Duplicados de _id eliminados. Nuevo shape: (21516901, 8)
Total de registros duplicados en '_id' después de limpieza: 0


## Valores fuera de rango o invalidos

In [15]:
# 1. Montos <= 0
mask_amount_non_positive = df['transaction_amount'] <= 0
count_amount_non_positive = mask_amount_non_positive.sum()
pct_amount_non_positive = count_amount_non_positive / len(df) * 100
print(f"❌ Montos ≤ 0: {count_amount_non_positive} filas")

❌ Montos ≤ 0: 0 filas


In [16]:
# 2. Fechas fuera de rango

fecha_min = df['transaction_date'].min()
fecha_max = df['transaction_date'].max()
print(f"📅 Fecha mínima: {fecha_min}")
print(f"📅 Fecha máxima: {fecha_max}")
mask_date_before = df['transaction_date'] < fecha_min
mask_date_after  = df['transaction_date'] > fecha_max
count_date_bad = (mask_date_before | mask_date_after).sum()
pct_date_bad = count_date_bad / len(df) * 100
print(f"❌ Fechas fuera de [{fecha_min}] / [{fecha_max}]: {count_date_bad} filas")

📅 Fecha mínima: 2021-01-01 00:00:40
📅 Fecha máxima: 2021-11-30 23:59:49
❌ Fechas fuera de [2021-01-01 00:00:40] / [2021-11-30 23:59:49]: 0 filas


## IDs Inesperados

In [17]:
# Verificar IDs inesperados o con formato incorrecto
print("\n🔍 VERIFICANDO IDs INESPERADOS")
print("=" * 60)

# Verificar longitud estándar de los IDs
id_lengths = df['_id'].str.len()
length_counts = id_lengths.value_counts().sort_index()
print(f"📏 Distribución de longitudes de IDs:")
for length, count in length_counts.items():
  pct = count / len(df) * 100
  print(f"   Longitud {length}: {count:,} IDs ({pct:.2f}%)")

# Verificar si todos los IDs son hexadecimales (solo caracteres 0-9, a-f)
pattern_hex = r'^[0-9a-f]+$'
mask_not_hex = ~df['_id'].str.match(pattern_hex, na=False)
count_not_hex = mask_not_hex.sum()
print(f"\n🔤 IDs que NO son hexadecimales: {count_not_hex}")

if count_not_hex > 0:
  print("   Ejemplos de IDs no hexadecimales:")
  ejemplos_no_hex = df.loc[mask_not_hex, '_id'].head(5).tolist()
  for ejemplo in ejemplos_no_hex:
    print(f"     {ejemplo}")

# Verificar IDs con caracteres especiales o espacios
mask_special_chars = df['_id'].str.contains(r'[^0-9a-fA-F]', na=False)
count_special_chars = mask_special_chars.sum()
print(f"\n⚠️ IDs con caracteres especiales: {count_special_chars}")

if count_special_chars > 0:
  print("   Ejemplos de IDs con caracteres especiales:")
  ejemplos_especiales = df.loc[mask_special_chars, '_id'].head(5).tolist()
  for ejemplo in ejemplos_especiales:
    print(f"     '{ejemplo}'")


🔍 VERIFICANDO IDs INESPERADOS
📏 Distribución de longitudes de IDs:
   Longitud 32: 21,516,901 IDs (100.00%)

🔤 IDs que NO son hexadecimales: 0

⚠️ IDs con caracteres especiales: 0


## Exportar Dataset Limpio

Guardamos el DataFrame limpio y procesado para usarlo en el siguiente notebook de análisis descriptivo.

In [18]:
# Exportar el DataFrame limpio para usar en el siguiente notebook
from pathlib import Path

# Crear directorio de salida si no existe
output_path = Path("../outputs/")
output_path.mkdir(exist_ok=True)

# Guardar el DataFrame limpio como un archivo parquet
parquet_file = output_path / "df_clean.parquet"
df.to_parquet(parquet_file, index=False)

print("DATASET EXPORTADO EXITOSAMENTE")


DATASET EXPORTADO EXITOSAMENTE
